In [93]:
%matplotlib inline

import librosa
import soundfile as sf
import numpy as np
from hmmlearn import hmm
import matplotlib.pyplot as plt  
import os

STRIDE_SIZE = 10.
FRAME_SIZE = 25.
N_MFCC = 16

WINDOW = 3000
WINDOW_STRIDE = 30

def find_peaks(X, Y, iterations=1):
    out = list(zip(X, Y))
    for i in range(iterations):
        temp_out = []
        for j in range(len(out)):
            if (j-1) >= 0 and (j+1) < len(out):
                if out[j-1][1] < out[j][1] and out[j][1] < out[j+1][1]:
                    temp_out.append(out[j])
        out = temp_out
    return out

def truncate_peaks(peaks, thresh = 5):
    curr = peaks[0]
    keep = []
    for p in peaks:
        if abs(p[0] - curr[0]) >= thresh:
            keep.append(curr)
        curr = p
    return keep
    
def window_number_to_sec(win_num, win_len = WINDOW):
    stamp = win_num + (win_len // 2)
    return (FRAME_SIZE + STRIDE_SIZE * (stamp - 1)) / 1000.

def get_mfcc_features(song_data, sample_rate, stride_size = STRIDE_SIZE, frame_size = FRAME_SIZE):
    mfccs = librosa.feature.mfcc(song_data, sample_rate, 
                                 n_mfcc=N_MFCC,
                                 hop_length=int(STRIDE_SIZE / 1000. * sample_rate), 
                                 n_fft=int(FRAME_SIZE / 1000. * sample_rate))
    return mfccs

def load_song(song_file):
    data, samplerate = sf.read(song_file)
    data = data[:, 0]
    mfccs = get_mfcc_features(data, samplerate)
    mfccs = np.asarray(mfccs).T
    return mfccs

def continuous_prob_estimate(mfccs):
    low_score = 1e9
    low_score_index = -1

    X = []
    Y = []
    for i in range(0, np.shape(mfccs)[0], WINDOW_STRIDE):
        if np.shape(mfccs)[0] - WINDOW <= i:
            break
        model = hmm.GaussianHMM(n_components=1	)
        model = model.fit(mfccs[i:i+WINDOW//2], [WINDOW//2])
        curr_score = model.score(mfccs[i+WINDOW//2:i+WINDOW], [len(mfccs[i+WINDOW//2:i+WINDOW])])
        X.append(window_number_to_sec(i))
        Y.append(curr_score * -1)
        if curr_score < low_score:
            low_score = curr_score
            low_score_index = i
    return X, Y
            
def make_peak_plot(X, Y, n=3):
    peaks = find_peaks(X, Y, iterations=n)
    trunc_peaks = truncate_peaks(peaks)
    #trunc_peaks = peaks
    print(trunc_peaks)
    plt.plot(X, Y)
    # for p in peaks:
    #     plt.axvline(x=p[0], color="red")
    for p in trunc_peaks:
        plt.axvline(x=p[0], color="red")

    plt.show()

In [66]:
from pathlib import Path
home = str(Path.home())
SONG_DIR = home + "/Downloads/songdata/songdata/"

results = []

for filename in os.listdir(SONG_DIR):
    if ".wav" in filename:
        mfccs = load_song(SONG_DIR + filename)
        X, Y = continuous_prob_estimate(mfccs)
        peaks = find_peaks(X, Y, iterations=3)
        trunc_peaks = truncate_peaks(peaks)
        print(filename,"\n", trunc_peaks)
        results.append((filename, trunc_peaks))
    

Axwell_Ingrosso-On_My_Way_Dave_Winnel_Remix.wav 
 [(57.915, 119695.03009149962), (101.715, 129256.15394450353), (120.615, 99049.55471084728), (130.815, 101840.66168576993), (141.015, 102127.12157366502), (171.915, 107665.78641522767), (188.415, 114628.36761185803), (216.615, 99500.37207539416), (248.415, 134593.5658925903)]
Bastille - Good Grief (Don Diablo Remix).wav 
 [(23.415, 108973.7290647585), (52.215, 99294.37844496059), (85.515, 99225.94958886865), (117.615, 103169.45325070799), (134.415, 105730.61295289642), (169.515, 99488.21792066089), (201.315, 138645.9386885502)]
Ariana Grande - No Tears Left To Cry (Dark Intensity Remix).wav 
 [(31.815, 98495.69486033858), (47.115, 107975.81431946083), (63.015, 99506.86303140932), (78.015, 100230.42689203676), (88.815, 101664.04889807818), (126.015, 107791.69085281854), (140.415, 100715.90275816072), (155.715, 99535.65534156257), (165.915, 101083.33611056523), (217.215, 101377.04074029803), (227.715, 99028.4847516799), (257.415, 97801.381

The Weeknd ft. Daft Punk - Starboy (Stephen Murphy Remix).wav 
 [(30.315, 142167.71075420213), (45.915, 127529.32970467495), (60.915, 107012.70206732204), (90.915, 119126.29955081898), (153.915, 137294.57834782955), (169.815, 133354.81076134188), (184.815, 108282.41855279823), (214.515, 119681.85660566426), (261.615, 97226.96158893911)]
Cheat Codes (ft. Demi Lovato) - No Promises (Ashworth Remix).wav 
 [(31.215, 99547.10753282125), (44.415, 96917.68150969583), (63.015, 97485.95598524052), (77.115, 102267.69155524303), (95.715, 113922.3467168287), (114.615, 99639.99201170242), (146.715, 97265.37596465865), (160.515, 102174.51008838735)]
Michael Jackson _ Love Never Felt So Good (Fedde Le Grand Remix).wav 
 [(43.815, 100376.14921807012), (72.015, 95248.33591011456), (93.915, 85164.52303219838), (119.115, 182871.8877678987), (134.115, 110164.5736546309), (165.315, 104103.72660246278), (186.915, 84985.53143948245), (193.515, 85027.46021587643), (216.315, 103590.94259389448), (246.615, 1329

In [105]:
import csv
import pandas as pd

df = pd.read_csv("songdatatable.csv")
list.sort(results, key = lambda x : x[0])

detection_count = 0
drop_count = 0
for i in range(len(results)):
    drops = str(df.iloc[[i]]["Drops"][i]).split(", ")
    peaks = results[i][1]
    temp_detection_count = 0
    drops_found = [df.iloc[[i]]["Song Name"][i]]
    print(drops, peaks)
    for d in drops:
        for p in peaks:
            dval = float(d)
            pval = float(p[0])
            if pval >= dval - 6 and pval <= dval + 6:
                temp_detection_count += 1
                drops_found.append(d)
    if temp_detection_count < len(drops):
        print(drops_found)
        print(results[i][0])
    drop_count += len(drops)
    detection_count += temp_detection_count

print(detection_count / drop_count)

['94.0', '172.1', '233.1'] [(31.815, 98495.69486033858), (47.115, 107975.81431946083), (63.015, 99506.86303140932), (78.015, 100230.42689203676), (88.815, 101664.04889807818), (126.015, 107791.69085281854), (140.415, 100715.90275816072), (155.715, 99535.65534156257), (165.915, 101083.33611056523), (217.215, 101377.04074029803), (227.715, 99028.4847516799), (257.415, 97801.38110244565)]
['Ariana_Grande_-_No_Tears_Left_To_Cry_(Dark_Intensity_Remix)', '94.0', '233.1']
Ariana Grande - No Tears Left To Cry (Dark Intensity Remix).wav
['48.2', '170.1'] [(18.015, 102615.07305118529), (31.515, 103570.6385807544), (40.215, 104991.25401127162), (63.015, 93698.44160496049), (78.615, 92697.85999987196), (108.615, 123508.59939430332), (123.615, 105998.15546071525), (139.815, 96899.2604128488), (160.815, 103606.61133391192), (200.115, 93636.48555016812)]
['Ariana_Grande_-_Side_To_Side_(TRU_Concept_Remix_ft._Romany)']
Ariana Grande - Side To Side (TRU Concept Remix ft. Romany).wav
['93.8', '203.0'] [(

In [102]:
song = "Calvin Harris & Disciples - How Deep Is Your Love.wav"

mfccs = load_song(SONG_DIR + song)
X, Y = continuous_prob_estimate(mfccs)


In [103]:
make_peak_plot(X, Y, n=3)

[(29.115, 108729.46967956323), (45.915, 104452.71143593459), (62.415, 97092.34702334927), (78.615, 114170.31718506635), (108.915, 101436.31371068867), (125.115, 96993.23714487356), (140.415, 109776.73730604387), (160.515, 97031.15372606482), (168.615, 97470.473239071)]
